In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

plt.rcParams["figure.figsize"] = (20, 9)
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import os

In [2]:
number_of_blinks=[] # total number of blinks in the video
frame_no = [] # use to store the frame no where algorithm will identify the blink
df_alert = pd.read_csv(f"AM_video_29_EAR.csv")
df_alert = df_alert.loc[(df_alert["EAR"]!=0)] # remove rows where facemesh could not detect the face
df_alert = df_alert.reset_index() #reindexing to identify the blink duration
df_alert.drop(columns=["index"],axis=1,inplace=True) #after reset_index(), one additional column is added which does not required
df_alert["NewEMA"] = df_alert["NewEAR"].ewm(4).mean() #exponetial weighted average moving a
for i in range(df_alert.shape[0]):
    if df_alert["NewEMA"].iloc[i] < df_alert["NewEMA"].iloc[i-4]*0.85: 
        if len(frame_no) ==0 or (df_alert["FN"].iloc[i] - frame_no[-1] >2 ):
            frame_no.append(df_alert["FN"].iloc[i])
        else:
            if  (df_alert["NewEMA"].iloc[i] < df_alert["NewEMA"].iloc[i-1]):
                frame_no[-1] = df_alert["FN"].iloc[i]           
number_of_blinks.append(len(frame_no))
print("\n\n-----------------------No of blinks---------------------")
print("File Name: {}, : {}".format("01_0_EAR.csv",frame_no))
#frame_no.clear()
print(number_of_blinks)




-----------------------No of blinks---------------------
File Name: 01_0_EAR.csv, : [241, 319, 375, 495, 621, 766, 857, 951, 1064, 1188, 1334, 1452, 1559, 1671, 1772]
[15]


In [3]:
l1 = [0.22940939,0.21666580,0.18969122,0.16046853,0.15831733,0.03591776,0.23685960]
df = pd.DataFrame(l1)
df["EWM"] = df.ewm(4).mean()
df

,0,EWM
0,0.229409,0.229409
1,0.216666,0.222330
2,0.189691,0.208953
3,0.160469,0.192529
4,0.158317,0.182352
5,0.035918,0.142660
6,0.236860,0.166499


In [4]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go


fig = make_subplots(rows=1, cols=1)


fig.append_trace(
    go.Scatter(x = df_alert["FN"], y = df_alert["NewEAR"],  name = "EAR", mode = "lines"),
    row=1, col=1
)
fig.append_trace(
    go.Scatter(x = df_alert["FN"], y = df_alert["NewEMA"],  name = "EMA", mode = "lines"),
    row=1, col=1
)
fig.append_trace(
    go.Scatter(x = df_alert["FN"], y = [np.mean(df_alert["NewEMA"])] * df_alert.shape[0],  name = "EMA_MEAN", mode = "lines"),
    row=1, col=1
)

fig.update_layout(height=450, width=1550, title_text="EAR with EMA")
fig.show()

In [5]:
blink_duration = pd.DataFrame(columns=["Start_blink","End_blink","Blink_frame","No_of_frames"])

for i in range(len(frame_no)):
    start_frame, end_frame, blinked_frame = 0, 0, 0
    blinked_frame = frame_no[i]
    ear = df_alert["NewEAR"].loc[df_alert["FN"]==blinked_frame].iloc[0]
    #print("EAR:",ear)
    if i == 0:
        sf = 0
    else:
        sf = df_alert.index[df_alert["FN"]==frame_no[i-1]][0]
    if i == len(frame_no)-1:
        en = len(df_alert.index)
    else:
        en = df_alert.index[df_alert["FN"]==frame_no[i+1]][0]
        
    for j in range(df_alert.index[df_alert["FN"]==frame_no[i]][0],sf,-1):
        if ear>(df_alert["NewEAR"].iloc[j-1]):
            ear = df_alert["NewEAR"].iloc[j-1]       
        else:
            if ear<df_alert["NewEAR"].iloc[j-1]*0.90:
                start_frame = df_alert["FN"].iloc[j-1]
                break
            else:
                continue
                
    for j in range(df_alert.index[df_alert["FN"]==frame_no[i]][0],en):
        if ear<(df_alert["NewEAR"].iloc[j+1]*0.90) and df_alert["NewEAR"].iloc[j+1]>0.1:
            end_frame = df_alert["FN"].iloc[j+1]
            break
        else:
            if j == en-1 or j == df_alert.index[0]:
                end_frame = df_alert["FN"].iloc[j]
            else:
                continue
    blink_duration.loc[blink_duration.shape[0]] = [start_frame,end_frame,blinked_frame,(end_frame-start_frame)]

In [6]:
#df_alert.loc[df_alert["FN"]==12992]
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_rows', None)
blink_duration

,Start_blink,End_blink,Blink_frame,No_of_frames
0,237,242,241,5
1,315,320,319,5
2,371,376,375,5
3,493,496,495,3
4,617,622,621,5
5,764,767,766,3
6,854,858,857,4
7,950,952,951,2
8,1059,1065,1064,6
9,1186,1189,1188,3


In [7]:
blink_duration["No_of_frames"].mean()

4.4

In [8]:
(blink_duration["No_of_frames"].sum()/df_alert.shape[0])

0.03614457831325301

### Blink Detection Based on EAR

In [427]:
number_of_blinks=[]
df_alert = pd.read_csv(f"Dataset/04_5_EAR.csv")
#df_alert["EMA"] = df_alert["EAR"].ewm(8).mean()
#df_alert["NewEMA"] = df_alert["NewEAR"].ewm(8).mean()
frameno=0
prev_frame=0
frame_no = []
counter = 0
df_alert = df_alert.loc[(df_alert["EAR"]!=0)]
for i in range(df_alert.shape[0]):
    if df_alert["NewEAR"].iloc[i] < df_alert["NewEAR"].iloc[i-4]*0.70:
        frameno = df_alert["FN"].iloc[i]
        if (df_alert["NewEAR"].iloc[i]>df_alert["NewEAR"].iloc[i-1]): 
            counter+=1
            frameno = frameno-1
            frame_no.append(frameno)
        if len(frame_no)>2 and frame_no[-1] - frame_no[-2] == 1:
            counter-=1
            frame_no.pop()    
            #print("else block",prev_frame)
        #frame_no.append(prev_frame)            
number_of_blinks.append(counter)
print("File Name: {}, : {}".format("same_file",frame_no))
#frame_no.clear()
print("\n\n-----------------------No of blinks---------------------")
print(number_of_blinks)

File Name: same_file, : [60, 61, 70, 194, 229, 245, 298, 342, 461, 834, 880, 990, 1006, 1142, 1246, 1275, 1448, 1576, 1801, 1813, 1848, 2067, 2088, 2208, 2240, 2320, 2488, 2525, 2548, 2741, 2882, 2902, 3621, 3642, 3659, 3663, 3688, 3743, 3768, 3808, 3976, 4003, 4026, 4106, 4279, 4384, 4413, 4474, 4604, 4644, 4696, 5309, 5475, 5534, 5634, 5676, 5708, 5873, 5897, 5916, 5950, 6057, 6087, 6454, 6764, 6785, 6902, 7302, 7839, 8020, 9015, 9105, 9201, 9421, 9707, 9826, 10129, 10170, 10370, 10448, 10686, 10777, 10779, 10888, 10987, 11150, 11349, 11565, 11815, 11927, 11993, 12057, 12107, 12125, 12253, 12265, 12282, 12367, 12568, 12628, 12646, 12673, 12791, 12852, 13044, 13136, 13311, 13409, 13416, 13560, 13650, 13652, 13899, 13918, 13982, 14161, 14278, 14376, 14474, 14499, 14538, 14550, 14613, 14662, 14900, 14953, 15047, 15256, 15311, 15356, 15403, 15462, 15559, 15635, 15672, 15712, 15801, 15817, 15990, 16217, 16285, 16345, 16436, 16946, 17184, 17254, 17284, 17326, 17700, 17819, 17821, 17904, 18